# Model_v3 Max length variation

In [1]:
# Enable TF Eager execution ==> for decoder later
import tensorflow as tf
tfe = tf.contrib.eager
tfe.enable_eager_execution()
Modes = tf.estimator.ModeKeys


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



In [2]:
%%bash
pip freeze | grep tensor

mesh-tensorflow==0.0.5
tensor2tensor==1.12.0
tensorboard==1.12.2
tensorflow-estimator==1.13.0rc0
tensorflow-gpu==1.13.0rc1
tensorflow-metadata==0.9.0
tensorflow-probability==0.5.0


In [3]:
%%bash
DATA_DIR=./data
TMP_DIR=$DATA_DIR/tmp
rm -rf $DATA_DIR $TMP_DIR
mkdir -p $DATA_DIR $TMP_DIR
# Generate data
t2t-datagen \
  --t2t_usr_dir=./train \
  --problem="onlinereview" \
  --data_dir=$DATA_DIR \
  --tmp_dir=$TMP_DIR

/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
INFO:tensorflow:Importing user module train from path /tf/jupyter-code/code/t2t/new
INFO:tensorflow:Generating problems:
    onlinereview:
      * onlinereview
INFO:tensorflow:Generating d

In [5]:
%%bash
DATA_DIR=./data
OUTDIR=./trained_model
rm -rf $OUTDIR
t2t-trainer \
  --data_dir=./data \
  --t2t_usr_dir=./train \
  --problem="onlinereview" \
  --model=transformer \
  --hparams_set=onlinereviewhp \
  --output_dir=$OUTDIR --job-dir=$OUTDIR --train_steps=10

/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
INFO:tensorflow:Importing user module train from path /tf/jupyter-code/code/t2t/new
Instructions for updating:
When switching to tf.estimator.Estimator, use tf.estimator.RunConfig instead.

## Generate t2t data

#### When generate the t2t dataset by running "generate_problem.py", 
#### reviews that the summary contains exactly same sentences with review text are dropped out.

In [11]:
 # self-defined 'generate_problem.py' file in the same folder.
# define the directory for generated data
# import tensorflow as tf
# from tensor2tensor.utils import registry
# from tensor2tensor.models import transformer
# from tensor2tensor.data_generators import problem
# from tensor2tensor.data_generators import text_encoder
# from tensor2tensor.data_generators import text_problems
# from tensor2tensor.data_generators import generator_utils

# from tensor2tensor.data_generators import problem
# from tensor2tensor.data_generators import text_problems
# import generate_problem_new

from tensor2tensor.bin import t2t_datagen

import tensorflow as tf


def main():
    
    t2t_datagen.main(argv)

def main(_):
  usr_dir.import_usr_dir(FLAGS.t2t_usr_dir)

  # Calculate the list of problems to generate.
  problems = sorted(
      list(_SUPPORTED_PROBLEM_GENERATORS) + registry.list_base_problems() +
      registry.list_env_problems())
  for exclude in FLAGS.exclude_problems.split(","):
    if exclude:
      problems = [p for p in problems if exclude not in p]
  if FLAGS.problem and FLAGS.problem[-1] == "*":
    problems = [p for p in problems if p.startswith(FLAGS.problem[:-1])]
  elif FLAGS.problem and "," in FLAGS.problem:
    problems = [p for p in problems if p in FLAGS.problem.split(",")]
  elif FLAGS.problem:
    problems = [p for p in problems if p == FLAGS.problem]
  else:
    problems = []

  # Remove TIMIT if paths are not given.
  if getattr(FLAGS, "timit_paths", None):
    problems = [p for p in problems if "timit" not in p]
  # Remove parsing if paths are not given.
  if getattr(FLAGS, "parsing_path", None):
    problems = [p for p in problems if "parsing_english_ptb" not in p]

  if not problems:
    problems_str = "\n  * ".join(
        sorted(
            list(_SUPPORTED_PROBLEM_GENERATORS) +
            registry.list_base_problems() + registry.list_env_problems()))
    error_msg = ("You must specify one of the supported problems to "
                 "generate data for:\n  * " + problems_str + "\n")
    error_msg += ("TIMIT and parsing need data_sets specified with "
                  "--timit_paths and --parsing_path.")
    raise ValueError(error_msg)

  if not FLAGS.data_dir:
    FLAGS.data_dir = tempfile.gettempdir()
    tf.logging.warning(
        "It is strongly recommended to specify --data_dir. "
        "Data will be written to default data_dir=%s.", FLAGS.data_dir)
  FLAGS.data_dir = os.path.expanduser(FLAGS.data_dir)
  tf.gfile.MakeDirs(FLAGS.data_dir)

  tf.logging.info("Generating problems:\n%s" %
                  registry.display_list_by_prefix(problems, starting_spaces=4))
  if FLAGS.only_list:
    return
  for problem in problems:
    set_random_seed()

    if problem in _SUPPORTED_PROBLEM_GENERATORS:
      generate_data_for_problem(problem)
    elif problem in registry.list_base_problems():
      generate_data_for_registered_problem(problem)
    elif problem in registry.list_env_problems():
      generate_data_for_env_problem(problem)
    else:
      tf.logging.error("Problem %s is not a supported problem for datagen.",
                       problem)
    
    
    
if __name__ == "__main__":
    tf.logging.set_verbosity(tf.logging.INFO)
    tf.app.run()
    

USR_DIR= './train'
PROBLEM = 'onlinereview'
TMP_DIR = './tmp' 
DATA_LOC = './data' 

# Generated training data
t2t_problem = generate_problem_new.OnlineRevewProjectUSYD_new(PROBLEM)
t2t_problem.generate_data(DATA_LOC, TMP_DIR) 

INFO:tensorflow:Generating vocab file: ./data/vocab.online_revew_project_usyd_new.32768.subwords
INFO:tensorflow:Trying min_count 500
INFO:tensorflow:Iteration 0
INFO:tensorflow:Processing token bsecread100mbsecwrite] took 0.13096165657043457 seconds, consider setting Text2TextProblem.max_subtoken_length to a smaller value.
INFO:tensorflow:Processing token [davidsonrating] took 0.262683629989624 seconds, consider setting Text2TextProblem.max_subtoken_length to a smaller value.
INFO:tensorflow:Processing token [5871400010058714000101587140001025871400199158714001992587140019935871400299158714002992587140029935871400310058714008001587140080025871400800358714009100587140091015871400910258714012400587140124015871401240258714012403587140124045871401240558714012406a5871401340058714013401587140134025871401340358714013404587140134055871401340658714013407a58714014400587140144015871401440258714014403587140144045871401440558714014406a587140194005871401940158714019402587140194035871401940458714019

INFO:tensorflow:Processing token [converastion] took 1.0528688430786133 seconds, consider setting Text2TextProblem.max_subtoken_length to a smaller value.
INFO:tensorflow:vocab_size = 153103
INFO:tensorflow:Iteration 1
INFO:tensorflow:Processing token [forhime] took 0.13114404678344727 seconds, consider setting Text2TextProblem.max_subtoken_length to a smaller value.
INFO:tensorflow:Processing token [5871400010058714000101587140001025871400199158714001992587140019935871400299158714002992587140029935871400310058714008001587140080025871400800358714009100587140091015871400910258714012400587140124015871401240258714012403587140124045871401240558714012406a5871401340058714013401587140134025871401340358714013404587140134055871401340658714013407a58714014400587140144015871401440258714014403587140144045871401440558714014406a58714019400587140194015871401940258714019403587140194045871401940558714019406a5871410280058714112400587141124015871411240258714113400587141134015871411340258714114400587141144

INFO:tensorflow:Processing token [moresturdy] took 0.26235508918762207 seconds, consider setting Text2TextProblem.max_subtoken_length to a smaller value.
INFO:tensorflow:vocab_size = 57272
INFO:tensorflow:Iteration 2
INFO:tensorflow:Processing token [nanshinken] took 0.130964994430542 seconds, consider setting Text2TextProblem.max_subtoken_length to a smaller value.
INFO:tensorflow:Processing token [5871400010058714000101587140001025871400199158714001992587140019935871400299158714002992587140029935871400310058714008001587140080025871400800358714009100587140091015871400910258714012400587140124015871401240258714012403587140124045871401240558714012406a5871401340058714013401587140134025871401340358714013404587140134055871401340658714013407a58714014400587140144015871401440258714014403587140144045871401440558714014406a58714019400587140194015871401940258714019403587140194045871401940558714019406a58714102800587141124005871411240158714112402587141134005871411340158714113402587141144005871411440

INFO:tensorflow:Processing token [experiencesinstaloveanddevoteshimselftoonlyonewoman] took 0.2627594470977783 seconds, consider setting Text2TextProblem.max_subtoken_length to a smaller value.
INFO:tensorflow:vocab_size = 58117
INFO:tensorflow:Iteration 3
INFO:tensorflow:Processing token [readscoopandvile] took 0.13106560707092285 seconds, consider setting Text2TextProblem.max_subtoken_length to a smaller value.
INFO:tensorflow:Processing token [5871400010058714000101587140001025871400199158714001992587140019935871400299158714002992587140029935871400310058714008001587140080025871400800358714009100587140091015871400910258714012400587140124015871401240258714012403587140124045871401240558714012406a5871401340058714013401587140134025871401340358714013404587140134055871401340658714013407a58714014400587140144015871401440258714014403587140144045871401440558714014406a58714019400587140194015871401940258714019403587140194045871401940558714019406a58714102800587141124005871411240158714112402587141

INFO:tensorflow:Processing token [ehheheheheeheehehehehehehehehehehhehehhehehehehhehehehehehehhehhehehhehehehehehheehheefreaking] took 0.2635314464569092 seconds, consider setting Text2TextProblem.max_subtoken_length to a smaller value.
INFO:tensorflow:vocab_size = 58059
INFO:tensorflow:Trying min_count 750
INFO:tensorflow:Iteration 0
INFO:tensorflow:Processing token bsecread100mbsecwrite] took 0.13042259216308594 seconds, consider setting Text2TextProblem.max_subtoken_length to a smaller value.
INFO:tensorflow:Processing token [davidsonrating] took 0.26241588592529297 seconds, consider setting Text2TextProblem.max_subtoken_length to a smaller value.
INFO:tensorflow:Processing token [5871400010058714000101587140001025871400199158714001992587140019935871400299158714002992587140029935871400310058714008001587140080025871400800358714009100587140091015871400910258714012400587140124015871401240258714012403587140124045871401240558714012406a58714013400587140134015871401340258714013403587140134

...

INFO:tensorflow:Processing token [converastion] took 1.0505695343017578 seconds, consider setting Text2TextProblem.max_subtoken_length to a smaller value.
INFO:tensorflow:vocab_size = 104338
INFO:tensorflow:Iteration 1
INFO:tensorflow:Processing token [religionpartucularism] took 0.13107848167419434 seconds, consider setting Text2TextProblem.max_subtoken_length to a smaller value.
INFO:tensorflow:Processing token [5871400010058714000101587140001025871400199158714001992587140019935871400299158714002992587140029935871400310058714008001587140080025871400800358714009100587140091015871400910258714012400587140124015871401240258714012403587140124045871401240558714012406a5871401340058714013401587140134025871401340358714013404587140134055871401340658714013407a58714014400587140144015871401440258714014403587140144045871401440558714014406a58714019400587140194015871401940258714019403587140194045871401940558714019406a58714102800587141124005871411240158714112402587141134005871411340158714113402587141

INFO:tensorflow:Processing token [crosbystillsnashetc] took 0.2626457214355469 seconds, consider setting Text2TextProblem.max_subtoken_length to a smaller value.
INFO:tensorflow:vocab_size = 39462
INFO:tensorflow:Iteration 2
INFO:tensorflow:Processing token [grittybutintelligent] took 0.13153648376464844 seconds, consider setting Text2TextProblem.max_subtoken_length to a smaller value.
INFO:tensorflow:Processing token [5871400010058714000101587140001025871400199158714001992587140019935871400299158714002992587140029935871400310058714008001587140080025871400800358714009100587140091015871400910258714012400587140124015871401240258714012403587140124045871401240558714012406a5871401340058714013401587140134025871401340358714013404587140134055871401340658714013407a58714014400587140144015871401440258714014403587140144045871401440558714014406a58714019400587140194015871401940258714019403587140194045871401940558714019406a587141028005871411240058714112401587141124025871411340058714113401587141134025

INFO:tensorflow:Processing token [davisongales] took 0.26232028007507324 seconds, consider setting Text2TextProblem.max_subtoken_length to a smaller value.
INFO:tensorflow:Processing token [funkycute] took 0.5249416828155518 seconds, consider setting Text2TextProblem.max_subtoken_length to a smaller value.
INFO:tensorflow:vocab_size = 40121
INFO:tensorflow:Iteration 3
INFO:tensorflow:Processing token [proboseidecarlo] took 0.13114213943481445 seconds, consider setting Text2TextProblem.max_subtoken_length to a smaller value.
INFO:tensorflow:Processing token [5871400010058714000101587140001025871400199158714001992587140019935871400299158714002992587140029935871400310058714008001587140080025871400800358714009100587140091015871400910258714012400587140124015871401240258714012403587140124045871401240558714012406a5871401340058714013401587140134025871401340358714013404587140134055871401340658714013407a58714014400587140144015871401440258714014403587140144045871401440558714014406a587140194005871

INFO:tensorflow:Processing token [comepsont252520durabritestandardcapacitycartridgedpb00kdiu3fgrefpd] took 0.2626829147338867 seconds, consider setting Text2TextProblem.max_subtoken_length to a smaller value.
INFO:tensorflow:Processing token [designcutoutanything] took 0.5238473415374756 seconds, consider setting Text2TextProblem.max_subtoken_length to a smaller value.
INFO:tensorflow:vocab_size = 40055
INFO:tensorflow:Generating case 0.
INFO:tensorflow:Generating case 100000.
INFO:tensorflow:Generating case 200000.
INFO:tensorflow:Generating case 300000.
INFO:tensorflow:Generating case 400000.
INFO:tensorflow:Generating case 500000.
INFO:tensorflow:Generating case 600000.
INFO:tensorflow:Generating case 700000.
INFO:tensorflow:Generating case 800000.
INFO:tensorflow:Generating case 900000.
INFO:tensorflow:Generating case 1000000.
INFO:tensorflow:Generating case 1100000.
INFO:tensorflow:Generating case 1200000.
INFO:tensorflow:Generating case 1300000.
INFO:tensorflow:Generating case 14

KeyboardInterrupt: 

In [7]:
from train import problem
ende_problem = problem.onlinereview('onlinereview')

/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [8]:

# ende_problem = problems.problem('onlinereview')
# ende_problem

NameError: name 'problems' is not defined

NameError: name 'problem' is not defined

In [9]:
# USR_DIR= './train'
# PROBLEM = 'onlinereview'
# TMP_DIR = './tmp' 
from tensor2tensor.utils import registry
from tensor2tensor import problems

DATA_LOC = './data' 
ende_problem = problems.problem('onlinereview')
vocab_name = "./train/vocab.onlinereview.8192.subwords"

# Get the encoders from the problem
encoders = ende_problem.feature_encoders(DATA_LOC)

# Setup helper functions for encoding and decoding
def encode(input_str, output_str=None):
    inputs = encoders["inputs"].encode(input_str) + [1]  
    batch_inputs = tf.reshape(inputs, [1, -1, 1])  
    print("inputs",inputs)
    return {"inputs": batch_inputs}

def decode(integers):
    integers = list(np.squeeze(integers))
    if 1 in integers:
        integers = integers[:integers.index(1)]
        print("integers", integers)
        print("targets", encoders["targets"])
    return encoders["targets"].decode(np.squeeze(integers))

In [12]:
# Create hparams and the model
from tensor2tensor.utils import trainer_lib
from tensor2tensor.utils import registry
from tensor2tensor import models
from tensor2tensor.utils.trainer_lib import create_hparams
import json

MODEL = 'transformer'
HPARAMS = 'transformer_prepend'
hparams = trainer_lib.create_hparams(HPARAMS, data_dir=DATA_LOC, problem_name=ende_problem)
hparams.problem_hparams.was_reversed =False

model = registry.model(MODEL)(hparams, Modes.EVAL) #tensorflow_exp_fn

ckpt_path = tf.train.latest_checkpoint("./model_files/") #model.ckpt
ckpt_path

INFO:tensorflow:Setting T2TModel mode to 'eval'
INFO:tensorflow:Setting hparams.label_smoothing to 0.0
INFO:tensorflow:Setting hparams.layer_prepostprocess_dropout to 0.0
INFO:tensorflow:Setting hparams.relu_dropout to 0.0
INFO:tensorflow:Setting hparams.symbol_dropout to 0.0
INFO:tensorflow:Setting hparams.attention_dropout to 0.0
INFO:tensorflow:Setting hparams.dropout to 0.0


In [14]:
import numpy as np


#Restore and summarize
def summarize(inputs):
    encoded_inputs = encode(inputs)
    with tfe.restore_variables_on_create(ckpt_path):
        model_output = model.infer(encoded_inputs)["outputs"]
    return decode(model_output)

inputs = '''
the archetypical american novel features a solitary protagonist undertaking an odyssey in which heshe confronts both physical and moral challenges and through which heshe emerges with a renewed spirit, transformed by the crucible of confrontations with adversaries, both real and imagined  sara gruens engaging water for elephants is an eminently american work, set in the depths of the depression and featuring a brokenhearted young man whose unplanned existential leap of faith catapults him into a chaotic and unknown way of life
'''
# inputs = '''
# happy with purchase even though it came a lot later than expected.'''
outputs = summarize(inputs)

print("Inputs: %s" % inputs)
print("Outputs: %s" % outputs)

inputs [3669, 3686, 3687, 128, 2, 3667, 3650, 3665, 3660, 3663, 1559, 495, 149, 2844, 5, 1796, 3652, 3650, 3653, 3648, 3667, 3661, 3653, 3654, 3659, 3649, 3648, 3668, 3647, 3654, 3664, 3663, 3650, 950, 50, 3653, 3664, 3643, 3649, 3649, 3663, 3643, 3668, 12, 63, 3660, 3663, 57, 3665, 3653, 3654, 3662, 3650, 3653, 3654, 3648, 1018, 187, 3652, 3660, 3643, 3649, 3659, 3665, 3520, 4, 2330, 3665, 3660, 3667, 3656, 3656, 3663, 3654, 3661, 3663, 1018, 4, 115, 63, 3660, 3663, 57, 3663, 3655, 3663, 3650, 3661, 3663, 1018, 22, 5, 3650, 3663, 3654, 3663, 3645, 2973, 683, 3, 3648, 3650, 3667, 3654, 3649, 3662, 3653, 3650, 3655, 2973, 43, 2, 3665, 3650, 3647, 3665, 3659, 3666, 3656, 1281, 6, 3665, 3653, 3654, 3662, 3650, 3653, 3654, 3648, 3667, 3648, 3659, 3653, 3654, 1018, 22, 3667, 3664, 3646, 3663, 3650, 3649, 3667, 3650, 3659, 3663, 1018, 3, 187, 147, 4, 2615, 42, 427, 2714, 1272, 409, 18, 325, 8, 50, 3663, 3655, 3659, 3654, 3663, 3654, 3648, 3656, 3643, 3668, 495, 162, 3, 528, 12, 2, 3664, 3663

In [10]:
# ende_problem.generate_data(data_dir, tmp_dir)
example = tfe.Iterator(ende_problem.dataset(Modes.TRAIN, DATA_LOC)).next()
inputs = [int(x) for x in example["inputs"].numpy()] # Cast to ints.
targets = [int(x) for x in example["targets"].numpy()] # Cast to ints.

INFO:tensorflow:Reading data files from ./data/onlinereview-train*
INFO:tensorflow:partition: 0 num_data_files: 10
Instructions for updating:
Colocations handled automatically by placer.


/usr/local/lib/python3.5/dist-packages/numpy/lib/type_check.py:546: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  'a.item() instead', DeprecationWarning, stacklevel=1)


In [11]:
import numpy as np
# Example inputs as int-tensor.
print("Inputs, encoded:")
print(inputs)
print("Inputs, decoded:")
# Example inputs as a sentence.
print(decode(inputs))

print("---"*10)
# Example targets as int-tensor.
print("Targets, encoded:")
print(targets)
# Example targets as a sentence.
print("Targets, decoded:")
print(decode(targets))

Inputs, encoded:
[275, 3524, 3243, 8, 50, 386, 407, 6, 362, 165, 3, 32, 49, 7, 1291, 109, 1444, 2, 1318, 6, 2, 294, 687, 57, 23, 5, 199, 6, 10, 2, 145, 25, 85, 622, 12, 2995, 4, 3066, 3, 63, 67, 24, 50, 3531, 3528, 29, 76, 2927, 6, 2, 47, 10, 44, 164, 3, 2, 145, 25, 967, 4, 5, 151, 130, 2093, 3, 63, 155, 44, 40, 531, 1050, 14, 730, 3, 2, 3617, 165, 25, 106, 183, 7, 28, 4, 2135, 10, 1228, 6, 109, 2846, 2665, 2139, 3, 306, 680, 2343, 2398, 3, 194, 24, 2, 1674, 2694, 2064, 3, 194, 1835, 1083, 3161, 363, 10, 1094, 2072, 3, 32, 1515, 2, 2569, 6, 935, 4, 1567, 4, 5, 171, 32, 478, 2, 3065, 6, 2, 66, 2576, 2903, 10, 179, 4, 793, 7, 2, 198, 3, 210, 6, 2, 165, 12, 15, 610, 8, 50, 386, 1876, 18, 77, 44, 259, 125, 2, 2454, 224, 71, 14, 1]
Inputs, decoded:
integers [275, 3524, 3243, 8, 50, 386, 407, 6, 362, 165, 3, 32, 49, 7, 1291, 109, 1444, 2, 1318, 6, 2, 294, 687, 57, 23, 5, 199, 6, 10, 2, 145, 25, 85, 622, 12, 2995, 4, 3066, 3, 63, 67, 24, 50, 3531, 3528, 29, 76, 2927, 6, 2, 47, 10, 44, 164, 3,

In [ ]:
# Generated 3637767 Examples

## Build and Train model

'dfsefs'

## Max len full

In [21]:
from tensor2tensor import models
from tensor2tensor.utils import registry, trainer_lib, decoding
from tensor2tensor.utils.trainer_lib import create_hparams
from tensor2tensor.utils.trainer_lib import create_hparams, create_run_config, create_experiment
import json

MODEL = 'transformer'
HPARAMS = 'transformer_prepend'
TRAIN_DIR = 'model_f'

### Define Hyper params and Train model

In [22]:
from tensor2tensor.utils.trainer_lib import create_hparams
import json
# Define hparams
hparams = create_hparams(HPARAMS)
hparams.batch_size = 1024 #512 
hparams.learning_rate = 0.1 
hparams.num_encoder_layers = 2
hparams.num_hidden_layers = 6
hparams.num_decoder_layers = 2 
hparams.dropout =0.4
hparams.num_heads =16
hparams.attention_dropout = 0.5 ##
# hparams.max_input_seq_length = ????

decode_hp = decoding.decode_hparams()
decode_hp.return_beams =False
# decode_hp.beam_size = 4

# print out all Hparams
print(json.loads(hparams.to_json()))
print(json.loads(decode_hp.to_json()))

{'add_relative_to_values': False, 'parameter_attention_value_channels': 0, 'mlperf_mode': False, 'clip_grad_norm': 0.0, 'optimizer_adafactor_beta1': 0.0, 'shared_embedding_and_softmax_weights': True, 'layer_prepostprocess_dropout': 0.1, 'learning_rate': 0.1, 'initializer_gain': 1.0, 'optimizer_zero_grads': False, 'self_attention_type': 'dot_product', 'symbol_dropout': 0.0, 'eval_run_autoregressive': False, 'max_input_seq_length': 0, 'multiply_embedding_mode': 'sqrt_depth', 'num_hidden_layers': 6, 'activation_dtype': 'float32', 'multiproblem_max_target_length': -1, 'pos': 'timing', 'warm_start_from_second': '', 'scheduled_sampling_prob': 0.0, 'multiproblem_fixed_train_length': -1, 'moe_overhead_eval': 2.0, 'no_data_parallelism': False, 'optimizer_adafactor_memory_exponent': 0.8, 'multiproblem_schedule_max_examples': 10000000.0, 'optimizer_adam_beta1': 0.9, 'proximity_bias': False, 'video_num_target_frames': 1, 'multiproblem_vocab_size': -1, 'scheduled_sampling_warmup_steps': 50000, 'nbr

### Build and train model

In [23]:
# create run config
t2t_problem = problem.onlinereview('onlinereview')

RUN_CONFIG = create_run_config(
    model_name=MODEL,
    model_dir=TRAIN_DIR,
    save_checkpoints_steps=1000,
    )

t2t_model = create_experiment(
        run_config=RUN_CONFIG,
        hparams=hparams,
        decode_hparams=decode_hp,
        model_name=MODEL,
        problem_name=t2t_problem, 
        data_dir='./data', 
        train_steps=100,
        eval_steps=100,
        min_eval_frequency=1000,
    )

# hparams.problem_hparams.was_reversed =False
print('\n hparams.problem_hparams \n',hparams.problem_hparams)

# training
t2t_model.train_and_evaluate()


INFO:tensorflow:Configuring DataParallelism to replicate the model.
INFO:tensorflow:schedule=continuous_train_and_eval
INFO:tensorflow:worker_gpu=1
INFO:tensorflow:sync=False
INFO:tensorflow:datashard_devices: ['gpu:0']
INFO:tensorflow:caching_devices: None
INFO:tensorflow:ps_devices: ['gpu:0']
INFO:tensorflow:Using config: {'_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7effd4550cc0>, '_is_chief': True, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_tf_random_seed': None, '_task_id': 0, '_task_type': None, '_eval_distribute': None, 'data_parallelism': <tensor2tensor.utils.expert_utils.Parallelism object at 0x7effd4550cf8>, 'use_tpu': False, '_num_ps_replicas': 0, 't2t_device_info': {'num_async_replicas': 1}, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_evaluation_master': '', '_model_dir': 'model_f', '_master': '', '_keep_checkpoint_max': 20, '_session_config': gpu_options {
  per_process_gpu_memory

/usr/local/lib/python3.5/dist-packages/tensorflow/python/platform/tf_logging.py:161: DeprecationWarning: The 'warn' method is deprecated, use 'warning' instead
  get_logger().warn(msg, *args, **kwargs)
/usr/local/lib/python3.5/dist-packages/numpy/lib/type_check.py:546: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  'a.item() instead', DeprecationWarning, stacklevel=1)
/usr/local/lib/python3.5/dist-packages/numpy/lib/type_check.py:546: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  'a.item() instead', DeprecationWarning, stacklevel=1)
/usr/local/lib/python3.5/dist-packages/numpy/lib/type_check.py:546: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  'a.item() instead', DeprecationWarning, stacklevel=1)


Instructions for updating:
Use `for ... in dataset:` to iterate over a dataset. If using `tf.estimator`, return the `Dataset` object directly from your input function. As a last resort, you can use `tf.compat.v1.data.make_initializable_iterator(dataset)`.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Setting T2TModel mode to 'train'
INFO:tensorflow:Using variable initializer: uniform_unit_scaling
INFO:tensorflow:Transforming feature 'inputs' with symbol_modality_3692_512.bottom
Instructions for updating:
Shapes are always computed; don't use the compute_shapes as it has no effect.
INFO:tensorflow:Transforming feature 'targets' with symbol_modality_3692_512.targets_bottom
INFO:tensorflow:Building model body
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use keras.layers.dense instead.
INFO:tensorflow:Transforming body output with symbol_modality_3692_512.top
INFO:tensorflow:Base learning r

/usr/local/lib/python3.5/dist-packages/tensorflow/python/ops/gradients_impl.py:110: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/usr/local/lib/python3.5/dist-packages/tensorflow/python/ops/gradients_impl.py:110: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into model_f/model.ckpt.


ResourceExhaustedError: OOM when allocating tensor with shape[2,16,304,304] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node transformer/parallel_0_5/transformer/transformer/body/decoder/layer_0/self_attention/multihead_attention/dot_product_attention/attention_weights (defined at /usr/local/lib/python3.5/dist-packages/tensor2tensor/layers/common_attention.py:1468) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

	 [[node transformer/parallel_0_5/transformer/transformer/body/encoder/layer_1/self_attention/multihead_attention/dot_product_attention/Max (defined at /usr/local/lib/python3.5/dist-packages/tensor2tensor/layers/common_attention.py:1153) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.


Caused by op 'transformer/paralle

## Max len 1024

In [2]:
# select the transformer model

from tensor2tensor import models
from tensor2tensor.utils import registry, trainer_lib, decoding, metrics
from tensor2tensor.utils.trainer_lib import create_hparams
from tensor2tensor.utils.trainer_lib import create_hparams, create_run_config, create_experiment
import json
import timeit
# running time calculation
start = timeit.default_timer()

MODEL = 'transformer'
HPARAMS = 'transformer_prepend'
TRAIN_DIR = 'model_files_1024_new'

# Define hparams
hparams = create_hparams(HPARAMS)
hparams.batch_size = 1024 #512 
hparams.learning_rate = 0.1 
# hparams.num_encoder_layers = 2
hparams.num_hidden_layers = 6
# hparams.num_decoder_layers = 2 
hparams.dropout =0.2
hparams.max_input_seq_length = 1024
hparams.shared_embedding_and_softmax_weights = False ###????

decode_hp = decoding.decode_hparams()
decode_hp.return_beams = False
# decode_hp.beam_size = 2

# print out all Hparams
print(json.loads(hparams.to_json()))
print(json.loads(decode_hp.to_json()))

# create run config
RUN_CONFIG = create_run_config(
    model_name=MODEL,
    model_dir=TRAIN_DIR,
    save_checkpoints_steps=1000,
    )
# t2t_problem.eval_metrics()

t2t_model_1024_new = create_experiment(
        run_config=RUN_CONFIG,
        hparams=hparams,
        decode_hparams=decode_hp,
        model_name=MODEL,
        problem_name=t2t_problem, 
        data_dir='./data', 
        train_steps=40001,
        eval_steps=100,
        min_eval_frequency=1000,
        eval_use_test_set=True
    )

hparams.problem_hparams.was_reversed =False
print('\n hparams.problem_hparams \n',hparams.problem_hparams)

# training
t2t_model_1024_new.train_and_evaluate()

# running time check (running on 1GPU server)
stop = timeit.default_timer()
print('Time: {} s'.format(round(stop - start,2)))

{'multiproblem_max_target_length': -1, 'scheduled_sampling_gold_mixin_prob': 0.5, 'scheduled_sampling_prob': 0.0, 'activation_dtype': 'float32', 'optimizer_adafactor_memory_exponent': 0.8, 'optimizer_adafactor_factored': True, 'attention_value_channels': 0, 'optimizer_adafactor_beta2': 0.999, 'tpu_enable_host_call': False, 'learning_rate': 0.1, 'symbol_modality_skip_top': False, 'max_relative_position': 0, 'multiproblem_reweight_label_loss': False, 'optimizer_adam_epsilon': 1e-09, 'optimizer_zero_grads': False, 'eval_run_autoregressive': False, 'modality': {}, 'pretrained_model_dir': '', 'parameter_attention_value_channels': 0, 'filter_size': 2048, 'optimizer_adam_beta1': 0.9, 'optimizer_adafactor_clipping_threshold': 1.0, 'multiproblem_schedule_max_examples': 10000000.0, 'warm_start_from_second': '', 'learning_rate_decay_staircase': False, 'length_bucket_step': 1.1, 'optimizer_adafactor_beta1': 0.0, 'multiproblem_target_eval_only': False, 'optimizer_adafactor_multiply_by_parameter_sca

/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/usr/local/lib/python3.5/dist-packages/tensorflow/python/platform/tf_logging.py:161: DeprecationWarning: The 'warn' method is deprecated, use 'warning' instead
  get_logger().warn(msg, *args, **kwargs)


INFO:tensorflow:Using config: {'_tf_random_seed': None, '_session_config': gpu_options {
  per_process_gpu_memory_fraction: 0.95
}
allow_soft_placement: true
graph_options {
  optimizer_options {
    global_jit_level: OFF
  }
}
, '_task_type': None, '_model_dir': 'model_files_1024_new', '_eval_distribute': None, '_keep_checkpoint_max': 20, '_train_distribute': None, '_num_worker_replicas': 0, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_log_step_count_steps': 100, '_environment': 'local', '_save_summary_steps': 100, '_num_ps_replicas': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f4a18ea40f0>, 't2t_device_info': {'num_async_replicas': 1}, '_master': '', '_keep_checkpoint_every_n_hours': 10000, '_task_id': 0, '_protocol': None, '_save_checkpoints_secs': None, '_is_chief': True, '_save_checkpoints_steps': 1000, 'use_tpu': False, '_evaluation_master': '', 'data_parallelism': <tensor2tensor.utils.expert_utils.Parallelism ob

/usr/local/lib/python3.5/dist-packages/numpy/lib/type_check.py:546: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  'a.item() instead', DeprecationWarning, stacklevel=1)
/usr/local/lib/python3.5/dist-packages/numpy/lib/type_check.py:546: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  'a.item() instead', DeprecationWarning, stacklevel=1)


Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use `for ... in dataset:` to iterate over a dataset. If using `tf.estimator`, return the `Dataset` object directly from your input function. As a last resort, you can use `tf.compat.v1.data.make_initializable_iterator(dataset)`.
INFO:tensorflow:Calling model_fn.


...

/usr/local/lib/python3.5/dist-packages/numpy/lib/type_check.py:546: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  'a.item() instead', DeprecationWarning, stacklevel=1)


INFO:tensorflow:Setting T2TModel mode to 'eval'
INFO:tensorflow:Setting hparams.layer_prepostprocess_dropout to 0.0
INFO:tensorflow:Setting hparams.label_smoothing to 0.0
INFO:tensorflow:Setting hparams.symbol_dropout to 0.0
INFO:tensorflow:Setting hparams.relu_dropout to 0.0
INFO:tensorflow:Setting hparams.attention_dropout to 0.0
INFO:tensorflow:Setting hparams.dropout to 0.0
INFO:tensorflow:Using variable initializer: uniform_unit_scaling
INFO:tensorflow:Transforming feature 'inputs' with symbol_modality_40055_512.bottom
INFO:tensorflow:Transforming feature 'targets' with symbol_modality_40055_512.targets_bottom
INFO:tensorflow:Building model body
INFO:tensorflow:Transforming body output with symbol_modality_40055_512.top
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-05-30T09:02:49Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from model_files_1024_new/model.ckpt-9000
INFO:tensorflow:Running local_init_op.
INFO:tensor

/usr/local/lib/python3.5/dist-packages/numpy/lib/type_check.py:546: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  'a.item() instead', DeprecationWarning, stacklevel=1)


INFO:tensorflow:Setting T2TModel mode to 'eval'
INFO:tensorflow:Setting hparams.layer_prepostprocess_dropout to 0.0
INFO:tensorflow:Setting hparams.label_smoothing to 0.0
INFO:tensorflow:Setting hparams.symbol_dropout to 0.0
INFO:tensorflow:Setting hparams.relu_dropout to 0.0
INFO:tensorflow:Setting hparams.attention_dropout to 0.0
INFO:tensorflow:Setting hparams.dropout to 0.0
INFO:tensorflow:Using variable initializer: uniform_unit_scaling
INFO:tensorflow:Transforming feature 'inputs' with symbol_modality_40055_512.bottom
INFO:tensorflow:Transforming feature 'targets' with symbol_modality_40055_512.targets_bottom
INFO:tensorflow:Building model body
INFO:tensorflow:Transforming body output with symbol_modality_40055_512.top
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-05-30T09:06:25Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from model_files_1024_new/model.ckpt-10000
INFO:tensorflow:Running local_init_op.
INFO:tenso

KeyboardInterrupt: 

## Max len 512

In [6]:
from tensor2tensor import models
from tensor2tensor.utils import registry, trainer_lib, decoding, metrics
from tensor2tensor.utils.trainer_lib import create_hparams
from tensor2tensor.utils.trainer_lib import create_hparams, create_run_config, create_experiment
import json
import timeit
# running time calculation
start = timeit.default_timer()

MODEL = 'transformer'
HPARAMS = 'transformer_prepend'
TRAIN_DIR = 'model_files_512'

# Define hparams
hparams = create_hparams(HPARAMS)
hparams.batch_size = 1024 #512 
hparams.learning_rate = 0.1 
hparams.num_encoder_layers = 2
hparams.num_hidden_layers = 6
hparams.num_decoder_layers = 2 
hparams.dropout =0.2
hparams.max_input_seq_length = 

decode_hp = decoding.decode_hparams()
decode_hp.return_beams =True
decode_hp.beam_size = 4

# print out all Hparams
print(json.loads(hparams.to_json()))
print(json.loads(decode_hp.to_json()))

# create run config
RUN_CONFIG = create_run_config(
    model_name=MODEL,
    model_dir=TRAIN_DIR,
    save_checkpoints_steps=1000,
    )
# t2t_problem.eval_metrics()

t2t_model_512 = create_experiment(
        run_config=RUN_CONFIG,
        hparams=hparams,
        decode_hparams=decode_hp,
        model_name=MODEL,
        problem_name=t2t_problem, 
        data_dir='./data', 
        train_steps=40001,
        eval_steps=100,
        min_eval_frequency=1000,
    )

hparams.problem_hparams.was_reversed =False
print('\n hparams.problem_hparams \n',hparams.problem_hparams)

# training
t2t_model_512.train_and_evaluate()

# running time check (running on 1GPU server)
stop = timeit.default_timer()
print('Time: {} s'.format(round(stop - start,2)))

{'optimizer_adam_beta2': 0.98, 'factored_logits': False, 'learning_rate_decay_scheme': 'noam', 'scheduled_sampling_warmup_steps': 50000, 'learning_rate_decay_rate': 1.0, 'heads_share_relative_embedding': False, 'overload_eval_metric_name': '', 'prepend_mode': 'prepend_inputs_masked_attention', 'initializer_gain': 1.0, 'learning_rate_warmup_steps': 8000, 'tpu_enable_host_call': False, 'weight_dtype': 'float32', 'optimizer_adafactor_factored': True, 'ffn_layer': 'dense_relu_dense', 'summarize_vars': False, 'force_full_predict': False, 'multiproblem_mixing_schedule': 'constant', 'num_encoder_layers': 2, 'filter_size': 2048, 'min_length': 0, 'activation_dtype': 'float32', 'dropout': 0.2, 'learning_rate_minimum': None, 'moe_overhead_eval': 2.0, 'label_smoothing': 0.1, 'learning_rate_decay_staircase': False, 'optimizer_adafactor_memory_exponent': 0.8, 'optimizer_adafactor_beta1': 0.0, 'multiproblem_per_task_threshold': '', 'multiproblem_fixed_train_length': -1, 'optimizer_adafactor_clipping_

/usr/local/lib/python3.5/dist-packages/tensorflow/python/platform/tf_logging.py:161: DeprecationWarning: The 'warn' method is deprecated, use 'warning' instead
  get_logger().warn(msg, *args, **kwargs)
/usr/local/lib/python3.5/dist-packages/numpy/lib/type_check.py:546: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  'a.item() instead', DeprecationWarning, stacklevel=1)
/usr/local/lib/python3.5/dist-packages/numpy/lib/type_check.py:546: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  'a.item() instead', DeprecationWarning, stacklevel=1)
/usr/local/lib/python3.5/dist-packages/numpy/lib/type_check.py:546: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  'a.item() instead', DeprecationWarning, stacklevel=1)


INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Setting T2TModel mode to 'train'
INFO:tensorflow:Using variable initializer: uniform_unit_scaling
INFO:tensorflow:Transforming feature 'inputs' with symbol_modality_40385_512.bottom
INFO:tensorflow:Transforming feature 'targets' with symbol_modality_40385_512.targets_bottom
INFO:tensorflow:Building model body
INFO:tensorflow:Transforming body output with symbol_modality_40385_512.top
INFO:tensorflow:Base learning rate: 0.100000
INFO:tensorflow:Trainable Variables Total size: 35396096
INFO:tensorflow:Non-trainable variables Total size: 5
INFO:tensorflow:Using optimizer Adam


/usr/local/lib/python3.5/dist-packages/tensorflow/python/ops/gradients_impl.py:110: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/usr/local/lib/python3.5/dist-packages/tensorflow/python/ops/gradients_impl.py:110: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into model_files_512/model.ckpt.
INFO:tensorflow:loss = 11.188004, step = 0
INFO:tensorflow:global_step/sec: 5.50608
INFO:tensorflow:loss = 9.417504, step = 100 (18.163 sec)
INFO:tensorflow:global_step/sec: 6.03151
INFO:tensorflow:loss = 8.071609, step = 200 (16.579 sec)
INFO:tensorflow:global_step/sec: 6.18876
INFO:tensorflow:loss = 7.0966353, step = 300 (16.158 sec)
INFO:tensorflow:global_step/sec: 6.21046
INFO:tensorflow:loss = 6.769985, step = 400 (16.103 sec)
INFO:tensorflow:global_step/sec: 6.13414
INFO:tensorflow:loss = 6.444959, step = 500 (16.301 sec)
INFO:tensorflow:global_step/sec: 6.12722
INFO:tensorflow:loss = 6.295968, step = 600 (16.322 sec)
INFO:tensorflow:global_step/sec: 6.23412
INFO:tensorflow:loss = 5.855097, step = 700 (16.03

...

INFO:tensorflow:Setting T2TModel mode to 'eval'
INFO:tensorflow:Setting hparams.dropout to 0.0
INFO:tensorflow:Setting hparams.label_smoothing to 0.0
INFO:tensorflow:Setting hparams.layer_prepostprocess_dropout to 0.0
INFO:tensorflow:Setting hparams.attention_dropout to 0.0
INFO:tensorflow:Setting hparams.relu_dropout to 0.0
INFO:tensorflow:Setting hparams.symbol_dropout to 0.0
INFO:tensorflow:Using variable initializer: uniform_unit_scaling
INFO:tensorflow:Transforming feature 'inputs' with symbol_modality_40385_512.bottom
INFO:tensorflow:Transforming feature 'targets' with symbol_modality_40385_512.targets_bottom
INFO:tensorflow:Building model body
INFO:tensorflow:Transforming body output with symbol_modality_40385_512.top
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-05-29T21:00:14Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from model_files_512/model.ckpt-38000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow

/usr/local/lib/python3.5/dist-packages/numpy/lib/type_check.py:546: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  'a.item() instead', DeprecationWarning, stacklevel=1)


INFO:tensorflow:Setting T2TModel mode to 'eval'
INFO:tensorflow:Setting hparams.dropout to 0.0
INFO:tensorflow:Setting hparams.label_smoothing to 0.0
INFO:tensorflow:Setting hparams.layer_prepostprocess_dropout to 0.0
INFO:tensorflow:Setting hparams.attention_dropout to 0.0
INFO:tensorflow:Setting hparams.relu_dropout to 0.0
INFO:tensorflow:Setting hparams.symbol_dropout to 0.0
INFO:tensorflow:Using variable initializer: uniform_unit_scaling
INFO:tensorflow:Transforming feature 'inputs' with symbol_modality_40385_512.bottom
INFO:tensorflow:Transforming feature 'targets' with symbol_modality_40385_512.targets_bottom
INFO:tensorflow:Building model body
INFO:tensorflow:Transforming body output with symbol_modality_40385_512.top
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-05-29T21:03:55Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from model_files_512/model.ckpt-39000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow

/usr/local/lib/python3.5/dist-packages/numpy/lib/type_check.py:546: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  'a.item() instead', DeprecationWarning, stacklevel=1)


INFO:tensorflow:Setting T2TModel mode to 'eval'
INFO:tensorflow:Setting hparams.dropout to 0.0
INFO:tensorflow:Setting hparams.label_smoothing to 0.0
INFO:tensorflow:Setting hparams.layer_prepostprocess_dropout to 0.0
INFO:tensorflow:Setting hparams.attention_dropout to 0.0
INFO:tensorflow:Setting hparams.relu_dropout to 0.0
INFO:tensorflow:Setting hparams.symbol_dropout to 0.0
INFO:tensorflow:Using variable initializer: uniform_unit_scaling
INFO:tensorflow:Transforming feature 'inputs' with symbol_modality_40385_512.bottom
INFO:tensorflow:Transforming feature 'targets' with symbol_modality_40385_512.targets_bottom
INFO:tensorflow:Building model body
INFO:tensorflow:Transforming body output with symbol_modality_40385_512.top
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-05-29T21:07:41Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from model_files_512/model.ckpt-40000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow

## Max len 256

In [8]:
from tensor2tensor import models
from tensor2tensor.utils import registry, trainer_lib, decoding, metrics
from tensor2tensor.utils.trainer_lib import create_hparams
from tensor2tensor.utils.trainer_lib import create_hparams, create_run_config, create_experiment
import json
import timeit
# running time calculation
start = timeit.default_timer()

MODEL = 'transformer'
HPARAMS = 'transformer_prepend'
TRAIN_DIR = 'model_files_256'

# Define hparams
hparams = create_hparams(HPARAMS)
hparams.batch_size = 1024 #512 
hparams.learning_rate = 0.1 
hparams.num_encoder_layers = 2
hparams.num_hidden_layers = 6
hparams.num_decoder_layers = 2 
hparams.dropout =0.2
hparams.max_input_seq_length = 

decode_hp = decoding.decode_hparams()
decode_hp.return_beams =True
decode_hp.beam_size = 4

# print out all Hparams
print(json.loads(hparams.to_json()))
print(json.loads(decode_hp.to_json()))

# create run config
RUN_CONFIG = create_run_config(
    model_name=MODEL,
    model_dir=TRAIN_DIR,
    save_checkpoints_steps=1000,
    )
# t2t_problem.eval_metrics()

t2t_model_256 = create_experiment(
        run_config=RUN_CONFIG,
        hparams=hparams,
        decode_hparams=decode_hp,
        model_name=MODEL,
        problem_name=t2t_problem, 
        data_dir='./data', 
        train_steps=40001,
        eval_steps=100,
        min_eval_frequency=1000,
    )

hparams.problem_hparams.was_reversed =False
print('\n hparams.problem_hparams \n',hparams.problem_hparams)

# training
t2t_model_256.train_and_evaluate()

# running time check (running on 1GPU server)
stop = timeit.default_timer()
print('Time: {} s'.format(round(stop - start,2)))

{'optimizer_adam_beta2': 0.98, 'factored_logits': False, 'learning_rate_decay_scheme': 'noam', 'scheduled_sampling_warmup_steps': 50000, 'learning_rate_decay_rate': 1.0, 'heads_share_relative_embedding': False, 'overload_eval_metric_name': '', 'prepend_mode': 'prepend_inputs_masked_attention', 'initializer_gain': 1.0, 'learning_rate_warmup_steps': 8000, 'tpu_enable_host_call': False, 'weight_dtype': 'float32', 'optimizer_adafactor_factored': True, 'ffn_layer': 'dense_relu_dense', 'summarize_vars': False, 'force_full_predict': False, 'multiproblem_mixing_schedule': 'constant', 'num_encoder_layers': 2, 'filter_size': 2048, 'min_length': 0, 'activation_dtype': 'float32', 'dropout': 0.2, 'learning_rate_minimum': None, 'moe_overhead_eval': 2.0, 'label_smoothing': 0.1, 'learning_rate_decay_staircase': False, 'optimizer_adafactor_memory_exponent': 0.8, 'optimizer_adafactor_beta1': 0.0, 'multiproblem_per_task_threshold': '', 'multiproblem_fixed_train_length': -1, 'optimizer_adafactor_clipping_

/usr/local/lib/python3.5/dist-packages/tensorflow/python/platform/tf_logging.py:161: DeprecationWarning: The 'warn' method is deprecated, use 'warning' instead
  get_logger().warn(msg, *args, **kwargs)
/usr/local/lib/python3.5/dist-packages/numpy/lib/type_check.py:546: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  'a.item() instead', DeprecationWarning, stacklevel=1)
/usr/local/lib/python3.5/dist-packages/numpy/lib/type_check.py:546: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  'a.item() instead', DeprecationWarning, stacklevel=1)
/usr/local/lib/python3.5/dist-packages/numpy/lib/type_check.py:546: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  'a.item() instead', DeprecationWarning, stacklevel=1)


INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Setting T2TModel mode to 'train'
INFO:tensorflow:Using variable initializer: uniform_unit_scaling
INFO:tensorflow:Transforming feature 'inputs' with symbol_modality_40385_512.bottom
INFO:tensorflow:Transforming feature 'targets' with symbol_modality_40385_512.targets_bottom
INFO:tensorflow:Building model body
INFO:tensorflow:Transforming body output with symbol_modality_40385_512.top
INFO:tensorflow:Base learning rate: 0.100000
INFO:tensorflow:Trainable Variables Total size: 35396096
INFO:tensorflow:Non-trainable variables Total size: 5
INFO:tensorflow:Using optimizer Adam


/usr/local/lib/python3.5/dist-packages/tensorflow/python/ops/gradients_impl.py:110: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/usr/local/lib/python3.5/dist-packages/tensorflow/python/ops/gradients_impl.py:110: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into model_files_256/model.ckpt.
INFO:tensorflow:loss = 10.968329, step = 0
INFO:tensorflow:global_step/sec: 5.51456
INFO:tensorflow:loss = 9.443371, step = 100 (18.136 sec)
INFO:tensorflow:global_step/sec: 6.10584
INFO:tensorflow:loss = 8.01007, step = 200 (16.377 sec)
INFO:tensorflow:global_step/sec: 6.13299
INFO:tensorflow:loss = 7.0976048, step = 300 (16.306 sec)
INFO:tensorflow:global_step/sec: 6.26872
INFO:tensorflow:loss = 6.5317473, step = 400 (15.952 sec)
INFO:tensorflow:global_step/sec: 6.25614
INFO:tensorflow:loss = 6.064748, step = 500 (15.984 sec)
INFO:tensorflow:global_step/sec: 6.15156
INFO:tensorflow:loss = 6.1698384, step = 600 (16.256 sec)
INFO:tensorflow:global_step/sec: 6.10149
INFO:tensorflow:loss = 5.9234796, step = 700 (16.

...

INFO:tensorflow:Setting T2TModel mode to 'eval'
INFO:tensorflow:Setting hparams.dropout to 0.0
INFO:tensorflow:Setting hparams.label_smoothing to 0.0
INFO:tensorflow:Setting hparams.layer_prepostprocess_dropout to 0.0
INFO:tensorflow:Setting hparams.attention_dropout to 0.0
INFO:tensorflow:Setting hparams.relu_dropout to 0.0
INFO:tensorflow:Setting hparams.symbol_dropout to 0.0
INFO:tensorflow:Using variable initializer: uniform_unit_scaling
INFO:tensorflow:Transforming feature 'inputs' with symbol_modality_40385_512.bottom
INFO:tensorflow:Transforming feature 'targets' with symbol_modality_40385_512.targets_bottom
INFO:tensorflow:Building model body
INFO:tensorflow:Transforming body output with symbol_modality_40385_512.top
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-05-29T23:31:13Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from model_files_256/model.ckpt-38000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow

/usr/local/lib/python3.5/dist-packages/numpy/lib/type_check.py:546: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  'a.item() instead', DeprecationWarning, stacklevel=1)


INFO:tensorflow:Setting T2TModel mode to 'eval'
INFO:tensorflow:Setting hparams.dropout to 0.0
INFO:tensorflow:Setting hparams.label_smoothing to 0.0
INFO:tensorflow:Setting hparams.layer_prepostprocess_dropout to 0.0
INFO:tensorflow:Setting hparams.attention_dropout to 0.0
INFO:tensorflow:Setting hparams.relu_dropout to 0.0
INFO:tensorflow:Setting hparams.symbol_dropout to 0.0
INFO:tensorflow:Using variable initializer: uniform_unit_scaling
INFO:tensorflow:Transforming feature 'inputs' with symbol_modality_40385_512.bottom
INFO:tensorflow:Transforming feature 'targets' with symbol_modality_40385_512.targets_bottom
INFO:tensorflow:Building model body
INFO:tensorflow:Transforming body output with symbol_modality_40385_512.top
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-05-29T23:34:53Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from model_files_256/model.ckpt-39000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow

/usr/local/lib/python3.5/dist-packages/numpy/lib/type_check.py:546: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  'a.item() instead', DeprecationWarning, stacklevel=1)


INFO:tensorflow:Setting T2TModel mode to 'eval'
INFO:tensorflow:Setting hparams.dropout to 0.0
INFO:tensorflow:Setting hparams.label_smoothing to 0.0
INFO:tensorflow:Setting hparams.layer_prepostprocess_dropout to 0.0
INFO:tensorflow:Setting hparams.attention_dropout to 0.0
INFO:tensorflow:Setting hparams.relu_dropout to 0.0
INFO:tensorflow:Setting hparams.symbol_dropout to 0.0
INFO:tensorflow:Using variable initializer: uniform_unit_scaling
INFO:tensorflow:Transforming feature 'inputs' with symbol_modality_40385_512.bottom
INFO:tensorflow:Transforming feature 'targets' with symbol_modality_40385_512.targets_bottom
INFO:tensorflow:Building model body
INFO:tensorflow:Transforming body output with symbol_modality_40385_512.top
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-05-29T23:38:35Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from model_files_256/model.ckpt-40000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow

# -------------- ###########-------------

In [124]:
import generate_problem # self-defined 'generate_problem.py' file in the same folder.

# define the directory for generated data
USR_DIR= './'
PROBLEM = 'OnlineRevewProjectUSYD_v4'
TMP_DIR = './tmp' # Where data files from internet stored
DATA_LOC = './data' # Where pre-prcessed data is stored

# Generated training data
t2t_problem = generate_problem.OnlineRevewProjectUSYD(PROBLEM)
t2t_problem.generate_data(DATA_LOC, TMP_DIR) 

INFO:tensorflow:Found vocab file: ./data/vocab.online_revew_project_usyd.32768.subwords
INFO:tensorflow:Skipping generator because outputs files exists at ['./data/online_revew_project_usyd-unshuffled-train-00000-of-00009', './data/online_revew_project_usyd-unshuffled-train-00001-of-00009', './data/online_revew_project_usyd-unshuffled-train-00002-of-00009', './data/online_revew_project_usyd-unshuffled-train-00003-of-00009', './data/online_revew_project_usyd-unshuffled-train-00004-of-00009', './data/online_revew_project_usyd-unshuffled-train-00005-of-00009', './data/online_revew_project_usyd-unshuffled-train-00006-of-00009', './data/online_revew_project_usyd-unshuffled-train-00007-of-00009', './data/online_revew_project_usyd-unshuffled-train-00008-of-00009', './data/online_revew_project_usyd-unshuffled-dev-00000-of-00001']
INFO:tensorflow:Skipping shuffle because output files exist


HParams([('batch_size_multiplier', 1), ('input_space_id', 0), ('loss_multiplier', 1.0), ('modality', {'targets': <tensor2tensor.layers.modalities.SymbolModality object at 0x7f585caada58>, 'inputs': <tensor2tensor.layers.modalities.SymbolModality object at 0x7f585caada90>}), ('stop_at_eos', 1), ('target_space_id', 0), ('vocabulary', {'targets': <tensor2tensor.data_generators.text_encoder.SubwordTextEncoder object at 0x7f585ce47710>, 'inputs': <tensor2tensor.data_generators.text_encoder.SubwordTextEncoder object at 0x7f585ce47710>}), ('was_copy', False), ('was_reversed', False)])

In [70]:
from tensor2tensor.utils import decoding
from tensor2tensor.utils import registry
from tensor2tensor import models
from tensor2tensor.utils.trainer_lib import create_hparams
from tensor2tensor.utils.trainer_lib import create_run_config, create_experiment
import tensorflow as tf
import json

MODEL = 'transformer'
HPARAMS = 'transformer_prepend'
TRAIN_DIR = 'model_files_256'

hparams = create_hparams(HPARAMS, data_dir=DATA_LOC, problem_name=t2t_problem)

decode_to_file = 'decoding_result'
output_dir = './'
ckpt_path = tf.train.latest_checkpoint("./model_files_1024/") #model.ckpt


In [161]:
hparams = create_hparams(HPARAMS, data_dir=DATA_LOC, problem_name=t2t_problem)
hparams

HParams([('activation_dtype', 'float32'), ('add_relative_to_values', False), ('attention_dropout', 0.1), ('attention_dropout_broadcast_dims', ''), ('attention_key_channels', 0), ('attention_value_channels', 0), ('attention_variables_3d', False), ('batch_shuffle_size', 512), ('batch_size', 4096), ('causal_decoder_self_attention', True), ('clip_grad_norm', 0.0), ('compress_steps', 0), ('conv_first_kernel', 3), ('daisy_chain_variables', True), ('data_dir', './data'), ('dropout', 0.2), ('eval_drop_long_sequences', False), ('eval_run_autoregressive', False), ('factored_logits', False), ('ffn_layer', 'dense_relu_dense'), ('filter_size', 2048), ('force_full_predict', False), ('grad_noise_scale', 0.0), ('heads_share_relative_embedding', False), ('hidden_size', 512), ('initializer', 'uniform_unit_scaling'), ('initializer_gain', 1.0), ('kernel_height', 3), ('kernel_width', 1), ('label_smoothing', 0.1), ('layer_postprocess_sequence', 'da'), ('layer_prepostprocess_dropout', 0.1), ('layer_prepostpr

# real decoding

In [162]:
from tensor2tensor.utils import trainer_lib, decoding
from tensor2tensor.bin import t2t_trainer

# t2t_trainer

MODEL = 'transformer'
HPARAMS = 'transformer_prepend'
TRAIN_DIR = 'model_files_256'

ckpt_path = tf.train.latest_checkpoint("./model_files_1024/") #model.ckpt
# BEAM_SIZE=4
# ALPHA=0.6

# data_dir=DATA_LOC
# problem = t2t_problem
# model = MODEL
# hparams_set = HPARAMS
# output_dir = TRAIN_DIR

# decode_to_file = 'decode_from_file'
#########
t2t_problem.get_hparams().was_reversed =False
####
hparams = create_hparams(HPARAMS, data_dir=DATA_LOC, problem_name=t2t_problem)

####
# hparams = registry.hparams('transformer_base_single_gpu')
# hparams.add_hparam("data_dir", DATA_LOC)
# t2t_problem.get_hparams().was_reversed =False
# hparams = t2t_problem.get_hparams(hparams)
# hparams
######


def create_decode_hparams():
    decode_hp = decoding.decode_hparams()
    decode_hp.shards = 1 #tf.flags.FLAGS.decode_shards
#     decode_hp.shard_id = flags.FLAGS.worker_id
    decode_in_memory = decode_hp.decode_in_memory
    decode_hp.decode_in_memory = decode_in_memory
    decode_hp.return_beams =True
    decode_hp.beam_size = 2
#     decode_hp.decode_to_file = FLAGS.decode_to_file
#     decode_hp.decode_reference = FLAGS.decode_reference
    return decode_hp
decode_hp = create_decode_hparams()

RUN_CONFIG = create_run_config(
    model_name=MODEL,
    model_dir=TRAIN_DIR,)

estimator = trainer_lib.create_estimator(
    MODEL,
    hparams,
    RUN_CONFIG, #t2t_trainer.create_run_config(hparams),
    decode_hparams=decode_hp)


INFO:tensorflow:Configuring DataParallelism to replicate the model.
INFO:tensorflow:schedule=continuous_train_and_eval
INFO:tensorflow:worker_gpu=1
INFO:tensorflow:sync=False
INFO:tensorflow:datashard_devices: ['gpu:0']
INFO:tensorflow:caching_devices: None
INFO:tensorflow:ps_devices: ['gpu:0']
INFO:tensorflow:Using config: {'_eval_distribute': None, '_master': '', 't2t_device_info': {'num_async_replicas': 1}, '_save_summary_steps': 100, 'data_parallelism': <tensor2tensor.utils.expert_utils.Parallelism object at 0x7f585c98e470>, '_keep_checkpoint_every_n_hours': 10000, '_device_fn': None, '_num_worker_replicas': 0, '_session_config': gpu_options {
  per_process_gpu_memory_fraction: 0.95
}
allow_soft_placement: true
graph_options {
  optimizer_options {
    global_jit_level: OFF
  }
}
, '_environment': 'local', '_is_chief': True, '_num_ps_replicas': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f585c98e668>, '_protocol': None, 'use_tpu': False, '_ke

/usr/local/lib/python3.5/dist-packages/tensorflow/python/platform/tf_logging.py:161: DeprecationWarning: The 'warn' method is deprecated, use 'warning' instead
  get_logger().warn(msg, *args, **kwargs)


In [163]:
hparams.problem_hparams

HParams([('batch_size_multiplier', 1), ('input_space_id', 0), ('loss_multiplier', 1.0), ('modality', {'targets': <tensor2tensor.layers.modalities.SymbolModality object at 0x7f585caada58>, 'inputs': <tensor2tensor.layers.modalities.SymbolModality object at 0x7f585caada90>}), ('stop_at_eos', 1), ('target_space_id', 0), ('vocabulary', {'targets': <tensor2tensor.data_generators.text_encoder.SubwordTextEncoder object at 0x7f585ce47710>, 'inputs': <tensor2tensor.data_generators.text_encoder.SubwordTextEncoder object at 0x7f585ce47710>}), ('was_copy', False), ('was_reversed', False)])

In [123]:
t2t_problem.get_hparams()

HParams([('batch_size_multiplier', 1), ('input_space_id', 0), ('loss_multiplier', 1.0), ('modality', {'targets': <tensor2tensor.layers.modalities.SymbolModality object at 0x7f597c0ba2e8>, 'inputs': <tensor2tensor.layers.modalities.SymbolModality object at 0x7f597c0ba320>}), ('stop_at_eos', 1), ('target_space_id', 0), ('vocabulary', {'targets': <tensor2tensor.data_generators.text_encoder.SubwordTextEncoder object at 0x7f599c039e48>, 'inputs': <tensor2tensor.data_generators.text_encoder.SubwordTextEncoder object at 0x7f599c039e48>}), ('was_copy', False), ('was_reversed', True)])

In [122]:
get_feature_encoders(DATA_LOC)

NameError: name 'get_feature_encoders' is not defined

In [8]:
import tensorflow as tf
ckpt_path = tf.train.latest_checkpoint("./model_files_1024_new/") #model.ckpt
TRAIN_DIR = 'model_files_1024_new'
RUN_CONFIG = create_run_config(
    model_name=MODEL,
    model_dir=TRAIN_DIR,)
estimator = trainer_lib.create_estimator(
    MODEL,
    hparams,
    RUN_CONFIG, #t2t_trainer.create_run_config(hparams),
    decode_hparams=decode_hp)

INFO:tensorflow:Configuring DataParallelism to replicate the model.
INFO:tensorflow:schedule=continuous_train_and_eval
INFO:tensorflow:worker_gpu=1
INFO:tensorflow:sync=False
INFO:tensorflow:datashard_devices: ['gpu:0']
INFO:tensorflow:caching_devices: None
INFO:tensorflow:ps_devices: ['gpu:0']
INFO:tensorflow:Using config: {'t2t_device_info': {'num_async_replicas': 1}, '_num_ps_replicas': 0, '_keep_checkpoint_every_n_hours': 10000, 'use_tpu': False, '_num_worker_replicas': 0, '_device_fn': None, '_train_distribute': None, '_task_id': 0, '_tf_random_seed': None, '_save_checkpoints_secs': None, '_evaluation_master': '', '_model_dir': 'model_files_1024_new', '_task_type': None, '_save_checkpoints_steps': 1000, '_environment': 'local', '_is_chief': True, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_keep_checkpoint_max': 20, 'data_parallelism': <tensor2tensor.utils.expert_utils.Parallelism object at 0x7ff4c7ba6a20>, '_session_config': gpu_options {
  per_process

/usr/local/lib/python3.5/dist-packages/tensorflow/python/platform/tf_logging.py:161: DeprecationWarning: The 'warn' method is deprecated, use 'warning' instead
  get_logger().warn(msg, *args, **kwargs)


In [4]:
hparams.problem_hparams

HParams([('batch_size_multiplier', 1), ('input_space_id', 0), ('loss_multiplier', 1.0), ('modality', {'targets': <tensor2tensor.layers.modalities.SymbolModality object at 0x7f4a18f65fd0>, 'inputs': <tensor2tensor.layers.modalities.SymbolModality object at 0x7f4a18ea4048>}), ('stop_at_eos', 1), ('target_space_id', 0), ('vocabulary', {'targets': <tensor2tensor.data_generators.text_encoder.SubwordTextEncoder object at 0x7f4a19fbd978>, 'inputs': <tensor2tensor.data_generators.text_encoder.SubwordTextEncoder object at 0x7f4a19fbd978>}), ('was_copy', False), ('was_reversed', False)])

In [6]:
ckpt_path = tf.train.latest_checkpoint("model_files_full_new")
ckpt_path #= './model_files_1024/model.ckpt-22000'

TRAIN_DIR = 'model_files_full_new'
RUN_CONFIG = create_run_config(
    model_name=MODEL,
    model_dir=TRAIN_DIR,)
estimator = trainer_lib.create_estimator(
    MODEL,
    hparams,
    RUN_CONFIG, #t2t_trainer.create_run_config(hparams),
    decode_hparams=decode_hp)

INFO:tensorflow:Configuring DataParallelism to replicate the model.
INFO:tensorflow:schedule=continuous_train_and_eval
INFO:tensorflow:worker_gpu=1
INFO:tensorflow:sync=False
INFO:tensorflow:datashard_devices: ['gpu:0']
INFO:tensorflow:caching_devices: None
INFO:tensorflow:ps_devices: ['gpu:0']
INFO:tensorflow:Using config: {'t2t_device_info': {'num_async_replicas': 1}, '_num_ps_replicas': 0, '_keep_checkpoint_every_n_hours': 10000, 'use_tpu': False, '_num_worker_replicas': 0, '_device_fn': None, '_train_distribute': None, '_task_id': 0, '_tf_random_seed': None, '_save_checkpoints_secs': None, '_evaluation_master': '', '_model_dir': 'model_files_full_new', '_task_type': None, '_save_checkpoints_steps': 1000, '_environment': 'local', '_is_chief': True, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_keep_checkpoint_max': 20, 'data_parallelism': <tensor2tensor.utils.expert_utils.Parallelism object at 0x7ff4c7ba6a90>, '_session_config': gpu_options {
  per_process

/usr/local/lib/python3.5/dist-packages/tensorflow/python/platform/tf_logging.py:161: DeprecationWarning: The 'warn' method is deprecated, use 'warning' instead
  get_logger().warn(msg, *args, **kwargs)


In [9]:
decoding.decode_interactively(estimator, hparams, decode_hp, checkpoint_path=ckpt_path)

AttributeError: 'HParams' object has no attribute 'problem'

In [21]:
review2 = '''
i origonally did not get the item i ordered.  when contacting the company they got back with me quickly.  it turned out the item was out of stock.  they sent it once it came in stock.  this was not a problem for me since i ordered the item way in advance for my daughters birthday.  i would recommend this selling but ordered ahead of time just incase.
'''
s2 = '''
happy with purchase even though it came a lot later than expected.
'''


In [18]:
# tf.estimator.ModeKeys.PREDICT

'infer'

In [26]:
inputs = review2

import numpy as np

vocab_name = "vocab.online_revew_project_usyd.32768.subwords"

# Get the encoders from the problem
encoders = t2t_problem.feature_encoders(DATA_LOC)

model_decoder = registry.model(MODEL)(hparams, Modes.EVAL)

def encode(input_str, output_str=None):
    inputs = encoders["inputs"].encode(input_str) + [1]  
    batch_inputs = tf.reshape(inputs, [1, -1, 1])  
    return {"inputs": batch_inputs}

encoded_inputs = encode(inputs)
with tfe.restore_variables_on_create(ckpt_path):
    model_output = model_decoder.infer(encoded_inputs)["outputs"]

Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Use tf.cast instead.


In [59]:
dataset_kwargs = {
  "shard": decode_hp.shards,
  "dataset_split": None,
  "max_records": 1}

In [ ]:
model_decoder = registry.model(MODEL)(hparams, Modes.EVAL)

In [47]:
infer_input_fn = t2t_problem.make_estimator_input_fn(
    tf.estimator.ModeKeys.PREDICT, hparams, dataset_kwargs=dataset_kwargs)

In [57]:
infer_input_fn = t2t_problem.make_estimator_input_fn(
    Modes.PREDICT, hparams, dataset_kwargs=dataset_kwargs)

In [58]:
# infer_input_fn = model_output

decoding.decode_once(estimator,
                t2t_problem,
                hparams,
                infer_input_fn,
                decode_hp,
                decode_to_file,
                output_dir,
                log_results=True,
                checkpoint_path=ckpt_path)

TypeError: predict() missing 1 required positional argument: 'input_fn'

In [ ]:
'''
  Args:
    estimator: tf.estimator.Estimator instance. Used to generate encoded
      predictions.
    problem_name: str. Name of problem.
    hparams: HParams instance. HParams for model training.
    infer_input_fn: zero-arg function. Input function for estimator.
    decode_hp: HParams instance. See decode_hparams() above.
    decode_to_file: str. Prefix for filenames. Used to generated filenames to
      which decoded predictions are written.
    output_dir: str. Output directory. Only used for writing images.
    log_results: bool. If False, return encoded predictions without any
      further processing.
    checkpoint_path: str. Path to load model checkpoint from. If unspecified,
      Estimator's default is used.
  Returns:
    If decode_hp.decode_in_memory is True:
      List of dicts, one per example. Values are either numpy arrays or decoded
      strings.
    If decode_hp.decode_in_memory is False:
      An empty list.
'''

In [4]:
# select the transformer model
from tensor2tensor.utils import registry
from tensor2tensor import models
from tensor2tensor.utils.trainer_lib import create_hparams
from tensor2tensor.utils.trainer_lib import create_run_config, create_experiment
import json
import timeit
# running time calculation
start = timeit.default_timer()

MODEL = 'transformer'
HPARAMS = 'transformer_prepend'
TRAIN_DIR = 'model_files_256'

hparams = create_hparams(HPARAMS)
hparams.batch_size = 1024 
hparams.learning_rate = 0.2 # default 0.2
hparams.num_encoder_layers = 6 # default 0
hparams.num_decoder_layers = 6 # default 0
hparams.max_input_seq_length = 256

RUN_CONFIG = create_run_config(
    model_name=MODEL,
    model_dir=TRAIN_DIR,)

t2t_model_256 = create_experiment(
        run_config=RUN_CONFIG,
        hparams=hparams,
        model_name=MODEL,
        problem_name=t2t_problem, #PROBLEM,
        data_dir='./data', 
        train_steps=40001, 
        eval_steps=200 
    )

# training
t2t_model_256.train_eval_and_decode()

# running time check (running on 1GPU server)
stop = timeit.default_timer()
print('Time: {} s'.format(round(stop - start,2)))

Instructions for updating:
When switching to tf.estimator.Estimator, use tf.estimator.RunConfig instead.
INFO:tensorflow:Configuring DataParallelism to replicate the model.
INFO:tensorflow:schedule=continuous_train_and_eval
INFO:tensorflow:worker_gpu=1
INFO:tensorflow:sync=False
INFO:tensorflow:datashard_devices: ['gpu:0']
INFO:tensorflow:caching_devices: None
INFO:tensorflow:ps_devices: ['gpu:0']


/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/usr/local/lib/python3.5/dist-packages/tensorflow/python/platform/tf_logging.py:161: DeprecationWarning: The 'warn' method is deprecated, use 'warning' instead
  get_logger().warn(msg, *args, **kwargs)


INFO:tensorflow:Using config: {'_is_chief': True, 'data_parallelism': <tensor2tensor.utils.expert_utils.Parallelism object at 0x7fdd24ae2080>, '_session_config': gpu_options {
  per_process_gpu_memory_fraction: 0.95
}
allow_soft_placement: true
graph_options {
  optimizer_options {
    global_jit_level: OFF
  }
}
, '_task_type': None, '_train_distribute': None, '_master': '', '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, 't2t_device_info': {'num_async_replicas': 1}, '_device_fn': None, '_evaluation_master': '', '_eval_distribute': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fdd24ae20b8>, '_protocol': None, '_environment': 'local', '_model_dir': 'model_files_256', '_num_worker_replicas': 0, 'use_tpu': False, '_tf_random_seed': None, '_save_checkpoints_steps': 1000, '_task_id': 0, '_num_ps_replicas': 0, '_keep_checkpoint_max': 20, '_keep_checkpoint_every_n_hours': 10000, '_save_summary_steps': 100, '_log_step_count_steps

/usr/local/lib/python3.5/dist-packages/numpy/lib/type_check.py:546: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  'a.item() instead', DeprecationWarning, stacklevel=1)
/usr/local/lib/python3.5/dist-packages/numpy/lib/type_check.py:546: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  'a.item() instead', DeprecationWarning, stacklevel=1)


Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use `for ... in dataset:` to iterate over a dataset. If using `tf.estimator`, return the `Dataset` object directly from your input function. As a last resort, you can use `tf.compat.v1.data.make_initializable_iterator(dataset)`.
INFO:tensorflow:Calling model_fn.


...

/usr/local/lib/python3.5/dist-packages/tensorflow/python/ops/gradients_impl.py:110: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/usr/local/lib/python3.5/dist-packages/tensorflow/python/ops/gradients_impl.py:110: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/usr/local/lib/python3.5/dist-packages/tensorflow/python/ops/gradients_impl.py:110: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/usr/local/lib/python3.5/dist-packages/tensorflow/python/ops/gradients_impl.py:110: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown

INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from model_files_256/model.ckpt-40001
Instructions for updating:
Use standard file utilities to get mtimes.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 40001 into model_files_256/model.ckpt.
INFO:tensorflow:Reading data files from ./data/online_revew_project_usyd-dev*
INFO:tensorflow:partition: 0 num_data_files: 1
INFO:tensorflow:Calling model_fn.


/usr/local/lib/python3.5/dist-packages/numpy/lib/type_check.py:546: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  'a.item() instead', DeprecationWarning, stacklevel=1)


INFO:tensorflow:Setting T2TModel mode to 'eval'
INFO:tensorflow:Setting hparams.symbol_dropout to 0.0
INFO:tensorflow:Setting hparams.label_smoothing to 0.0
INFO:tensorflow:Setting hparams.layer_prepostprocess_dropout to 0.0
INFO:tensorflow:Setting hparams.attention_dropout to 0.0
INFO:tensorflow:Setting hparams.dropout to 0.0
INFO:tensorflow:Setting hparams.relu_dropout to 0.0
INFO:tensorflow:Using variable initializer: uniform_unit_scaling
INFO:tensorflow:Transforming feature 'inputs' with symbol_modality_40385_512.bottom
INFO:tensorflow:Transforming feature 'targets' with symbol_modality_40385_512.targets_bottom
INFO:tensorflow:Building model body
INFO:tensorflow:Transforming body output with symbol_modality_40385_512.top
Instructions for updating:
tf.py_func is deprecated in TF V2. Instead, use
    tf.py_function, which takes a python function which manipulates tf eager
    tensors instead of numpy arrays. It's easy to convert a tf eager tensor to
    an ndarray (just call tensor.n

KeyboardInterrupt: 

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

from tensor2tensor.bin import t2t_decoder

import tensorflow as tf

  --data_dir=$DATA_DIR \
  --problem=$PROBLEM \
  --model=$MODEL \
  --hparams_set=$HPARAMS \
  --output_dir=$TRAIN_DIR \
  --decode_hparams="beam_size=$BEAM_SIZE,alpha=$ALPHA" \
  --decode_from_file=$DECODE_FILE \
  --decode_to_file=translation.en

def main(argv):
    t2t_decoder.main(argv)

if __name__ == "__main__":
    tf.logging.set_verbosity(tf.logging.INFO)
    tf.app.run()

# Decoding

In [61]:
from tensor2tensor.data_generators import problem
from tensor2tensor.data_generators import text_problems
from tensor2tensor.utils import registry
from tensor2tensor import models
from tensor2tensor.utils.trainer_lib import create_hparams
import generate_problem
USR_DIR= './'
PROBLEM = 'OnlineRevewProjectUSYD_v4'
TMP_DIR = './tmp' 
DATA_LOC = './data' 

t2t_problem = generate_problem.OnlineRevewProjectUSYD(PROBLEM)
t2t_problem.generate_data(DATA_LOC, TMP_DIR) 

MODEL = 'transformer'
HPARAMS = 'transformer_prepend'
TRAIN_DIR = 'model_files_2014'

hparams = create_hparams(HPARAMS, data_dir=DATA_LOC, problem_name=t2t_problem)

INFO:tensorflow:Found vocab file: ./data/vocab.online_revew_project_usyd.32768.subwords
INFO:tensorflow:Skipping generator because outputs files exists at ['./data/online_revew_project_usyd-unshuffled-train-00000-of-00009', './data/online_revew_project_usyd-unshuffled-train-00001-of-00009', './data/online_revew_project_usyd-unshuffled-train-00002-of-00009', './data/online_revew_project_usyd-unshuffled-train-00003-of-00009', './data/online_revew_project_usyd-unshuffled-train-00004-of-00009', './data/online_revew_project_usyd-unshuffled-train-00005-of-00009', './data/online_revew_project_usyd-unshuffled-train-00006-of-00009', './data/online_revew_project_usyd-unshuffled-train-00007-of-00009', './data/online_revew_project_usyd-unshuffled-train-00008-of-00009', './data/online_revew_project_usyd-unshuffled-dev-00000-of-00001']
INFO:tensorflow:Skipping shuffle because output files exist


In [67]:
import numpy as np

vocab_name = "vocab.online_revew_project_usyd.32768.subwords"

# Get the encoders from the problem
encoders = t2t_problem.feature_encoders(DATA_LOC)

# Setup helper functions for encoding and decoding
def encode(input_str, output_str=None):
    inputs = encoders["inputs"].encode(input_str) + [1]  
    batch_inputs = tf.reshape(inputs, [1, -1, 1])  
    return {"inputs": batch_inputs}

def decode(integers):
    integers = list(np.squeeze(integers))
    if 1 in integers:
        integers = integers[:integers.index(1)]
    return encoders["inputs"].decode(np.squeeze(integers))

#Restore and summarize
def summarize(inputs):
    encoded_inputs = encode(inputs)
    with tfe.restore_variables_on_create(ckpt_path):
        model_output = model.infer(encoded_inputs)["outputs"]
    return decode(model_output)

In [66]:
model = registry.model(MODEL)(hparams, Modes.PREDICT)

In [68]:
ckpt_path = tf.train.latest_checkpoint("./model_files_1024/") #model.ckpt
ckpt_path

'./model_files_1024/model.ckpt-40001'

### Decoder example

In [69]:

review1 = '''
we have many of the old , old issue but the number had depleted there were not enough books to allow us to use them regularly with the additional supply the books will be used more often they are a good old standby for gospel singing
'''
review2 = '''
i origonally did not get the item i ordered.  when contacting the company they got back with me quickly.  it turned out the item was out of stock.  they sent it once it came in stock.  this was not a problem for me since i ordered the item way in advance for my daughters birthday.  i would recommend this selling but ordered ahead of time just incase.
'''
s2 = '''
happy with purchase even though it came a lot later than expected.
'''

summary = summarize(review2)

print("REVIEWS: %s" % review2)
print("PRED_SUMMARY: %s" % summary)
print("GOLD_SUMMARY: %s" % s2)


REVIEWS: 
i origonally did not get the item i ordered.  when contacting the company they got back with me quickly.  it turned out the item was out of stock.  they sent it once it came in stock.  this was not a problem for me since i ordered the item way in advance for my daughters birthday.  i would recommend this selling but ordered ahead of time just incase.

PRED_SUMMARY: read 10; i the the did not get the item i ordered.  when its company they got back with me me quickly.  it turned out the item was out of stock.  they sent it once it came in stock.  this was not a problem for me since i ordered the item way in advance for my daughters birthday.  i would recommend this selling but ordered ahead of time just incase the has 10;
GOLD_SUMMARY: 
happy with purchase even though it came a lot later than expected.

